In [1]:
import pandas as pd

import sys
sys.path.append('../')

# Data Collection

Let's use the ProFootballReferenceScraper to collect NFL player and team statistics.

In [2]:
from src.scraper import ProFootballReferenceScraper

scraper = ProFootballReferenceScraper(data_dir="../data")
scraper.scrape_years(start_year=1999, end_year=2024)

Scraping years:   0%|          | 0/26 [00:01<?, ?it/s]


KeyboardInterrupt: 

# Data Processing

Build the silver layer from the scraped data

In [2]:

from src.processor import FantasyDataProcessor

processor = FantasyDataProcessor(data_dir="../data", drop_rookies=True)
processor.process_all_data()

Processing files matching: *_player_fantasy_stats.csv: 100%|██████████| 25/25 [00:00<00:00, 284.02it/s]
2025-07-27 23:08:53,316 - src.processor - INFO - Saved player_fantasy_stats.csv to ../data/silver/player_fantasy_stats.csv
Processing files matching: *_player_receiving_stats.csv: 100%|██████████| 25/25 [00:00<00:00, 205.73it/s]
2025-07-27 23:08:59,281 - src.processor - INFO - Saved player_receiving_stats.csv to ../data/silver/player_receiving_stats.csv
Processing files matching: *_player_rushing_stats.csv: 100%|██████████| 25/25 [00:00<00:00, 210.38it/s]
2025-07-27 23:09:03,708 - src.processor - INFO - Saved player_rushing_stats.csv to ../data/silver/player_rushing_stats.csv
Processing files matching: *_player_passing_stats.csv:   0%|          | 0/25 [00:00<?, ?it/s]2025-07-27 23:09:03,714 - src.processor - INFO - Added missing pass_qbr column to ../data/bronze/2001_player_passing_stats.csv
2025-07-27 23:09:03,716 - src.processor - INFO - Added missing pass_qbr column to ../data/bro

## Feature Analysis 

- Do some data quality checks on the final stats
- Do some sanity checks on the final stats
- Look at which features are the most and least informative

In [3]:

from src.feature_engineering import FantasyFeatureEngineer

engineer = FantasyFeatureEngineer(
    data_dir="../data",
    metadata_cols=["id"],
    target_cols=["standard_fantasy_points", "standard_fantasy_points_per_game", "ppr_fantasy_points", "ppr_fantasy_points_per_game", "value_over_replacement"],
)
final_stats = engineer.gold_data

2025-07-27 23:09:30,180 - src.feature_engineering - INFO - Loaded gold table: 9071 rows


In [4]:
# See shape
print(final_stats.shape)

# Check null values
null_values = final_stats.isnull().sum()
print(f"Rows with null values: {null_values[null_values > 0].count()}")

# Check data types
data_types = final_stats.dtypes
print(f"Non float data types: {data_types[data_types != 'float64'].count()}")

# Check for duplicates
print(f"\nDuplicate rows in final stats: {final_stats.duplicated().sum()}")

(12749, 194)
Rows with null values: 0
Non float data types: 1

Duplicate rows in final stats: 0


In [5]:
# Check that games are joined correctly (rookies should have 0 game played)
print(final_stats[final_stats['id'] == 'malik_nabers_2024'][['id', 'age','games', 'games_2_yr_avg', 'games_3_yr_avg']])

# Check that the correct year in player fantasy stats is joined with the correct year in player stats

# Check that the last year of data is dropped

# TODO: brainstorm other sanity checks

Empty DataFrame
Columns: [id, age, games, games_2_yr_avg, games_3_yr_avg]
Index: []


In [14]:
# print rookie algorithm
display(final_stats[(final_stats['games'] == 0) & (final_stats['games_2yr_avg'] == 0) & (final_stats['games_3yr_avg'] == 0)])

,id,age,standard_fantasy_points,ppr_fantasy_points,value_over_replacement,standard_fantasy_points_per_game,ppr_fantasy_points_per_game,rec_targets,rec_receptions,rec_yards,...,team_plays_2_yr_avg,team_yards_per_play_2_yr_avg,team_points_3_yr_avg,team_yards_3_yr_avg,team_plays_3_yr_avg,team_yards_per_play_3_yr_avg,awards,games,games_2yr_avg,games_3yr_avg
4,matt_forte_2008,23.0,242.0,304.5,95.0,15.12,19.03,0.0,0.0,0.0,...,1038.50,4.75,340.33,4664.00,1004.67,4.63,0.0,0.0,0.0,0.0
5,adrian_peterson_2008,23.0,241.0,261.5,94.0,15.06,16.34,0.0,0.0,0.0,...,994.50,5.20,317.67,4978.33,978.00,5.10,0.0,0.0,0.0,0.0
12,steve_slaton_2008,22.0,222.0,271.9,76.0,13.88,16.99,0.0,0.0,0.0,...,961.50,5.10,302.00,4617.67,959.00,4.80,0.0,0.0,0.0,0.0
20,chris_johnson_2008,23.0,207.0,249.8,61.0,13.80,16.65,0.0,0.0,0.0,...,991.00,4.95,308.00,4973.00,1001.33,4.97,0.0,0.0,0.0,0.0
27,antonio_bryant_2008,27.0,167.0,250.0,50.0,10.44,15.62,0.0,0.0,0.0,...,973.50,4.90,281.67,4755.33,977.33,4.87,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12740,jmon_moore_2018,23.0,-1.0,1.5,0.0,-0.08,0.12,0.0,0.0,0.0,...,1014.00,5.30,373.33,5381.33,1028.00,5.23,0.0,0.0,0.0,0.0
12742,rod_streater_2018,30.0,-1.0,-1.1,0.0,-0.33,-0.37,0.0,0.0,0.0,...,995.50,5.00,258.67,5076.33,1011.00,5.03,0.0,0.0,0.0,0.0
12745,jj_jones_2018,26.0,-2.0,-0.7,0.0,-0.50,-0.18,0.0,0.0,0.0,...,1018.41,5.38,359.50,5537.81,1022.20,5.42,0.0,0.0,0.0,0.0
12746,kyle_lauletta_2018,23.0,-2.0,-2.2,0.0,-1.00,-1.10,0.0,0.0,0.0,...,1027.00,5.05,325.33,5423.67,1035.67,5.27,0.0,0.0,0.0,0.0


In [6]:

engineer.generate_feature_analysis()

## Modelling

- Start with linear regression
- Become more advanced
- Forward, Backward, Feature Selection
- Models per target

In [13]:
from src.modelling import FantasyModel

ppr_model = FantasyModel(data_dir="../data", target_col="ppr_fantasy_points")
ppr_ppg_model = FantasyModel(data_dir="../data", target_col="ppr_fantasy_points_per_game")
standard_model = FantasyModel(data_dir="../data", target_col="standard_fantasy_points")
standard_ppg_model = FantasyModel(data_dir="../data", target_col="standard_fantasy_points_per_game")
vor_model = FantasyModel(data_dir="../data", target_col="value_over_replacement")

In [16]:
data = standard_model.split_data()

eval_pipeline = standard_model.create_pipeline()
grid_search = standard_model.create_grid_search(eval_pipeline, data)

print(f"Best params: {grid_search.best_params_}")
print(f"Best score: {grid_search.best_score_}")
print(f"Best estimator: {grid_search.best_estimator_}")

Best params: {'model': Lasso(), 'select__k': 'all'}
Best score: 0.5608183033581228
Best estimator: Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', StandardScaler()),
                ('select',
                 SelectKBest(k='all',
                             score_func=<function f_regression at 0x133cbbce0>)),
                ('model', Lasso())])


In [17]:
# Get mean test score and std test score for each model and make a dataframe out of it
cv_results_df = pd.DataFrame(grid_search.cv_results_)
#display(cv_results_df.columns)

display(cv_results_df[['param_model', 'param_select__k', 'mean_test_score', 'std_test_score']].sort_values(by='mean_test_score', ascending=False))

,param_model,param_select__k,mean_test_score,std_test_score
8,Lasso(),all,0.560818,0.023811
5,Ridge(),all,0.552621,0.022876
11,RandomForestRegressor(),all,0.551653,0.026641
2,LinearRegression(),all,0.548334,0.022760
7,Lasso(),150,0.546662,0.025549
6,Lasso(),100,0.545870,0.025423
17,HistGradientBoostingRegressor(),all,0.544749,0.023840
10,RandomForestRegressor(),150,0.544274,0.025064
3,Ridge(),100,0.542942,0.027131
0,LinearRegression(),100,0.540187,0.027394
